In [1]:
import numpy as np
from kifmm_py import KiFmm

# Single Precision

In [2]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = 6 # Single expansion order
n_vec = 1
n_crit = 150
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"  # Or 'helmholtz'
field_translation = "blas"  # Or 'fft'
kernel_eval_type = (
    "eval"  # For potentials, or 'eval_deriv' for potentials and gradients
)

# Setup source/target/charge data in Fortran order
sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

fmm = KiFmm(
    expansion_order,
    sources,
    targets,
    charges,
    kernel_eval_type,
    kernel,
    field_translation,
    prune_empty=True, # Optional, remove empty leaves and their ancestors
    timed=True, # Optional, can examine individual operator times
    svd_threshold=1e-7,  # Optional, required for 'blas' based field translations
    n_crit=n_crit, # Optional, but must specify either n_crit or depth
    surface_diff=1 # Optional (for BLAS field translation), creates check surfaces 
                   # with an expansion order 'surface_diff' higher than the corresponding equivalent surface
)

# Run the FMM
fmm.evaluate()

# Examine result at leaf index
leaf = fmm.target_leaves[1]
found = fmm.potentials(leaf)[0]
targets_leaf = fmm.target_coordinates(leaf)
expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

# Test
# np.testing.assert_allclose(found, expected, rtol=1e-5)